In [2]:
import os
import pickle
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import Counter
from tqdm import tqdm

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix



In [47]:
import pandas as pd
from datetime import datetime, timedelta
import os

def separate_files(df, output_folder):
    # Convert 'post_time' column to datetime
    df['post_time'] = pd.to_datetime(df['post_time'])

    # Sort DataFrame by 'post_time'
    df = df.sort_values(by='post_time')

    # Remove rows where 'content' column is NaN
    df = df.dropna(subset=['content'])

    # Define the start date and end date
    start_date = df['post_time'].min()
    end_date = df['post_time'].max()

    folder_count = 1
    result = []

    while start_date < end_date:
        # Define the end date for the training period (four months)
        train_end_date = start_date + pd.DateOffset(months=4) - timedelta(days=3)
        # Define the start and end dates for the test period (one month)
        test_start_date = train_end_date + timedelta(days=1)
        test_end_date = test_start_date + pd.DateOffset(months=1) - timedelta(days=1)

        # Get the data for the current period
        train_data = df[(df['post_time'] >= start_date) & (df['post_time'] <= train_end_date)]
        test_data = df[(df['post_time'] >= test_start_date) & (df['post_time'] <= test_end_date)]

        # Get the unique ids for the training and test data
        train_unique_ids = len(train_data['id'].unique())
        test_unique_ids = len(test_data['id'].unique())

        # Format the file names
        train_period_str = start_date.strftime('%Y%m')
        train_file = f"{train_period_str}.csv"
        test_period_str = test_start_date.strftime('%Y%m')
        test_file = f"{test_period_str}test.csv"

        # Create folder if it doesn't exist
        folder_path = os.path.join(output_folder, f"Folder {folder_count}")
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Write data to CSV files
        train_file_path = os.path.join(folder_path, train_file)
        test_file_path = os.path.join(folder_path, test_file)
        train_data.to_csv(train_file_path, index=False)
        test_data.to_csv(test_file_path, index=False)

        # Append the statistics to the result list
        result.append({
            'Folder': folder_count,
            'Train File': train_file,
            'Train Unique IDs': train_unique_ids,
            'Test File': test_file,
            'Test Unique IDs': test_unique_ids
        })

        # Move to the next month
        start_date = start_date + pd.DateOffset(months=1)
        folder_count += 1

    # Convert the result list to a DataFrame
    result_df = pd.DataFrame(result)

    # Save the result as a CSV file
    result_file_path = os.path.join(output_folder, 'separation_result.csv')
    result_df.to_csv(result_file_path, index=False)

# Specify the input file path
input_file_path = r"C:\Users\bradl\Desktop\BDA Mid\bda-midterm\dataset\disc_df.csv"
# Specify the output folder
output_folder = r"C:\Users\bradl\Desktop\BDA Mid\bda-midterm\output"

# Read the CSV file
disc_df = pd.read_csv(input_file_path)

# Call the function to separate the files
separate_files(disc_df, output_folder)
